In [1]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from datetime import datetime as dt
from datetime import timezone as tz

from joserfc.jwk import ECKey
from joserfc import jws
import json
import time
import random

# AggregatedObservations topic in MQTT
eventsdn = 'events/down/pop/general' 

# Broker address
broker = 'mqtt.example.com'

# generate key pair for signing messages
certfile = ".tls/signer.pem" 
signkey = ".tls/signer-key.pem"

# Send Observation to Edge Policy Manager (POP)
def observation_packet(domain, tag):
    # Generate test packet
    # Create an mqtt message with a single domain
    # with all relevant tag bits set

    ts = dt.now(tz=tz.utc).isoformat()

    # Domain data
    #  Name: the domain name to act on
    #  TimeAdded: when the observation was made
    #  TTL: how long POP should act on this information
    #  TagMask: all attributes related to the domain namn
    #
    dom = {
       'Name' : domain,
       'TimeAdded': ts, 
       'TTL': 1200, 
       'TagMask': tag
      }

    # Domain data that is added to POP
    added = [ dom ]

    # Domain data that is deleted from POP
    deleted = []

    # MQTT message
    msg = {
       'SrcName': 'dns-tapir',
       'MsgType': 'observation',
       'ListType': 'greylist',
       'Added': added,
       'Deleted' : deleted,
       'Msg': '',
       'TimeStamp': ts,
       'Creator': 'spark.example.com'
      }

    # Message signature
    # Currently EC/P-256, should be signed with Core signature
    # POP only trusts data with proper signature
    member = {
           'protected': {'alg': 'ES256'},
         }

    with open(signkey) as keyf:
        privatekey = ECKey.import_key(keyf.read())

    payload = json.dumps(msg)
    out = json.dumps(jws.serialize_json(member, payload, privatekey))

    # Send a single MQTT event without creating a persistent client
    publish.single(eventsdn, out, qos=2, hostname=broker, protocol=mqtt.MQTTv5)


## Send a domain with a numerical flag field to TEM


In [ ]:
# Meta-tags
# -------------------
#
# NewDomain
# Resolving domain that is seen for the first time (in an x day window)
#
# LowRankDomain
# Metatag that indicates that the domain is low ranked according to 
# Tranco, OpenPageRank, DNS query statistics, etc. 
#
# PossibleMalware
# Metatag that is the result of aggregating a number of features, 
# correlations to other sources or other indicators, and is set if there
# is risk that the domain is related to malware. Includes all malware 
# interactions, such as download sites, call-home or C2, etc
# 
# GlobalRampup 
# General rapid growth of queries
#
# LocalRampUp
# Significant local growth of queries
#
# SustainedHighTraffic
# The volume of queries is remarkably higher than baseline
# 

NewDomain = 2
LowRankDomain = 4
PossibleMalware = 8
GlobalRampUp = 16
LocalRampUp = 32
SustainedHighTraffic = 64

# The domain the meesage relates to
domain = "example.com."

# Set specific tag value
# value = ( GlobalRampUp | SustainedHighTraffic | LowRankDomain )
# observation_packet(domain, value)

# Send a continuous stream of observations
while True:
    # Generate random tags
    value = random.randrange(65537)
    # Send packet
    observation_packet(domain, value)
    print(f"{dt.now()} sent {value} for {domain}")
    # Wait a while
    time.sleep(45)
    
    # Reset all tags
    value = 0
    # send packet
    observation_packet(domain, value)
    print(f"{dt.now()} sent {value} for {domain}")
    # wait a while
    time.sleep(15)
